In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/trainingset/train.csv
/kaggle/input/testwork/train.csv


In [ ]:
import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup --colab > setup_logs.txt

In [ ]:
!autotrain setup

INFO     | 2024-06-17 23:59:08 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2024-06-17 23:59:08 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers


In [ ]:
project_name = 'finetunellama'
model_name = 'meta-llama/Llama-2-7b'

In [ ]:
push_to_hub = True
hf_token = "insert token"
hf_username = "insert username"

In [ ]:
learning_rate = 2e-4
num_epochs = 1
batch_size = 1
block_size = 5
trainer = "sft"
warmup_ratio = 0.1
weight_decay = 0.01
gradient_accumulation = 4
mixed_precision = "fp16"
peft = True
quantization = "int4"
lora_r = 16
lora_alpha = 32
lora_dropout = 0.05

In [ ]:
import os
os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["MIXED_PRECISION"] = str(mixed_precision)
os.environ["PEFT"] = str(peft)
os.environ["QUANTIZATION"] = str(quantization)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)
os.environ["HF_USERNAME"] = hf_username
os.environ["TRAINER"] = trainer

In [ ]:
import os
print(os.listdir("../input"))

['trainingset', 'testwork']


In [ ]:
!autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path /kaggle/input/testwork \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--quantization ${QUANTIZATION} \
--mixed-precision ${MIXED_PRECISION} \
--username ${HF_USERNAME} \
--trainer ${TRAINER} \
$( [[ "$PEFT" == "True" ]] && echo "--peft" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN}" )

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "insert model path"

# Load the tokenizer and model from the specified path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype='auto'
).eval()

# Define the user message with a refined prompt
prompt = """
Write a concise git commit message for the following code change using this structure for the commit message:
- A short, descriptive title
- A more detailed description on a new line explaining:
  - What has been changed
  - Why the change was made
  - Any additional context or relevant information:

Use a professional tone, with a focus on clarity and completeness to improve understanding for future reference.

Insert code change.

Commit message:
"""

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors='pt')

# Generate a response from the model
input_ids = input_ids['input_ids'].to('cuda')
output_ids = model.generate(input_ids, max_length=1000, temperature=0.7, top_p=0.9, num_return_sequences=1)
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the model's response
print("Model response:", response.strip())
